![EUROPE](http://nkpxv44v81c37wr753vdgve1.wpengine.netdna-cdn.com/wp-content/uploads/2015/10/Fotolia_44254321_Subscription_XXL-1000x250.jpg)

# WHERE TO RETIRE AT 50?

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

# Table of contents
1. [Research](#research)
2. [Some paragraph](#paragraph1)
    1. [Sub paragraph](#subparagraph1)
3. [Another paragraph](#paragraph2)


## Imports/Setup <a name="introduction"></a>

In [51]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

## RESEARCH <a name="research"></a>

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla at tellus dictum, dignissim nisi vel, molestie felis. Donec suscipit ullamcorper elit, at hendrerit neque eleifend a. Quisque orci erat, laoreet eget quam eget, accumsan porttitor leo. Sed imperdiet dolor eget accumsan posuere. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Donec purus ligula, ullamcorper quis enim ac, sodales pretium arcu. Nullam ut quam facilisis sapien fermentum pharetra. Nulla facilisi.

___

## BOKEH EXAMPLE

In [45]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
reset_output()
output_notebook()

Loading BokehJS ...

In [81]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import output_file, show

#the data
xList=[0.0, 445277, 556597, 667916, 779236, 890555]
yList=[6446275, 4163881, 5465442, 6800125, 6621293, 6446275]
source=ColumnDataSource({'x':xList,'y':yList})

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)")
]

x_range=(min(xList),max(xList))
y_range=(min(yList),max(yList))
plot = figure(title='Europe',tooltips=TOOLTIPS, tools= "pan,wheel_zoom",x_range=x_range,y_range=y_range,width=900, height=600)#create a figure
plot.add_tile(CARTODBPOSITRON)#add the CARTODBPOSITRON background tile

#plot a dot an a line
plot.circle(source=source,x='x',y='y', size=10, color ='blue')#circle

show(plot)


In [109]:
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON

#the data
xList=[0.0, 556597, 667916, 779236, 890555]
yList=[6446275, 5465442, 6800125, 6621293, 6446275]
source=ColumnDataSource({'x':xList,'y':yList})

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
]

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-1000000, 3000000), y_range=(4000000, 9000000),
           x_axis_type="mercator", y_axis_type="mercator",width=900, height=600, tooltips=TOOLTIPS)
p.add_tile(CARTODBPOSITRON)

#plot a dot an a line
p.circle(source=source,x='x',y='y', size=10, color ='blue')#circle

show(p)